### Script
- **Input:** real-time querying on ConceptNet5 number-batch in the Neo4j Graph database.
- **Output:** Micro-tags(concepts) generated for listing amenities and stored in a redis like memcache locally.
              
- **Note:** 
    - This is a workaround to minimize the response time in a real-time conversation. 
    - The mem cache can be synchronized in a distributed system on data-event triggers.
    - The micro-tags(related concepts) will be stored as dictionary values with the actual
      amenity as keys and used for enhaned search for matched listings(item attributes) in the Neo4j Graph DB. 


In [1]:
!pip install py2neo
from py2neo import Graph
import re
import pandas as pd

# use neo4j for real-time recommendations.
g = Graph("bolt://localhost:7687/neo4j", password = "test")

In [2]:
df = pd.read_csv('./Data/processing/Neo4j/Amenity.csv')
amenities = df['name'].unique()
len(amenities)

889

In [3]:
dic = dict()

def realTimeRecommendation(amenity,topK):		
	concepts=""

	query = """
							CALL ga.nlp.ml.word2vec.nn($wid, $k, 'en-ConceptNet5') YIELD word, distance RETURN word AS list;
							"""
	for row in g.run(query, wid=amenity, k=topK):
		processed=re.sub('[^a-zA-Z0-9]+', ' ', row[0])
		concepts+=str(processed)+","
	
	print(amenity+" concepts cached")
	amenity=str(amenity)
	dic[amenity] = dic.get(amenity, [])
	dic[amenity].append(concepts.rstrip(',')) 

for amenity in amenities:
	try:
		amen = amenity.split(' ')
		for a in amen:
			realTimeRecommendation(str(a.lower()),10)
	except:
		print('no split character')
		realTimeRecommendation(str(amenity.lower()),10)

    

refrigerator concepts cached
paid concepts cached
parking concepts cached
off concepts cached
premises concepts cached
long concepts cached
term concepts cached
stays concepts cached
allowed concepts cached
paid concepts cached
washer concepts cached
– concepts cached
in concepts cached
unit concepts cached
lock concepts cached
on concepts cached
bedroom concepts cached
door concepts cached
heating concepts cached
drying concepts cached
rack concepts cached
for concepts cached
clothing concepts cached
hair concepts cached
dryer concepts cached
body concepts cached
soap concepts cached
room-darkening concepts cached
shades concepts cached
mini concepts cached
fridge concepts cached
shampoo concepts cached
first concepts cached
aid concepts cached
kit concepts cached
clothing concepts cached
storage: concepts cached
wardrobe concepts cached
hot concepts cached
water concepts cached
laundromat concepts cached
nearby concepts cached
mosquito concepts cached
net concepts cached
portable con

In [4]:
dic

{'refrigerator': ['refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge',
  'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge',
  'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge',
  'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge',
  'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge',
  'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge',
  'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge',
  'refriger

In [5]:
import json
with open('RASA_ConceptNet5/mem_cache_conceptNet5.json', 'w') as outfile:
    json.dump(dic,outfile)

In [6]:
outfile.close()

In [7]:
data={}
with open('RASA_ConceptNet5/mem_cache_conceptNet5.json') as json_file:
    data = json.load(json_file)


if 'refrigerator' in data:
    print(data['refrigerator'])

['refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge', 'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge', 'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge', 'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge', 'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge', 'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge', 'refrigerator,minirefrigerator,keep food cold,keeping food cold,electric refrigerator,ice safe,frigerator,cool food,fridge,microfridge', 'refrigerator,minirefrigerator,keep food

In [8]:
import string 
strP = '{"image":"https://a0.muscache.com/pictures/10272854/8dcca016_original.jpg"}'
strP = strP.strip("'")
strP

import json
dic={}
dic['a'] = 1
dic['b'] = 2
aList=[]
aList.append(dic)

jsonStr = json.dumps(aList)
print(jsonStr)

[{"a": 1, "b": 2}]
